In [1]:
import requests
import pandas as pd

In [2]:
url = 'https://scanr.enseignementsup-recherche.gouv.fr/api/scanr-publications/_search'

In [5]:
params = {'size': 1000,
 'query': {'bool': {'must': [{'query_string': {'query': '"3D integration"',
      'fields': ['id^10',
       'externalIds.id^3',
       'title.*^3',
       'summary.*^2',
       'domains.label.*^2',
       'domains.code^2',
       'authors.fullName^2',
       'authors.denormalized.id^3',
       'authors.denormalized.orcid^3',
       'authors.denormalized.id_hal^3',
       'authors.denormalized.idref^3',
       'authors.affiliations.ids.id',
       'authors.affiliations.name',
       'source.title',
       'source.journalIssns',
       'source.publisher',
       'affiliations.id',
       'affiliations.label.*',
       'softwares.softwareName',
       'softwares.contexts',
       'softwares.wikidata',
       'projects.id',
       'projects.label.*',
       'projects.acronym.*']}}]}},
 'aggs': {'byYear': {'terms': {'field': 'year', 'size': 40}},
  'byPublicationType': {'terms': {'field': 'type.keyword'}},
  'byAuthors': {'terms': {'field': 'authors.id_name.keyword', 'size': 10}},
  'byIsOa': {'terms': {'field': 'isOa'}},
  'byFunder': {'terms': {'field': 'projects.type.keyword'}},
  'byReview': {'terms': {'field': 'source.title.keyword', 'size': 10}}}}

In [7]:
r = requests.post(url, json=params, verify=False).json()

/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scanr.enseignementsup-recherche.gouv.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:
x = r['hits']['hits']

In [9]:
len(x)

432

In [34]:
oa_data = []

In [43]:
for ix, e in enumerate(x):
    if ix%10==0:
        print(ix, end=',')
    if 'doi' not in e['_source']['id']:
        print(e['_source']['id'])
        continue
    doi = e['_source']['id'].replace('doi', '')
    r_oa = requests.get(f'https://api.openalex.org/works?filter=doi:{doi}').json()['results'][0]
    oa_data.append(r_oa)


0,halin2p3-00846062
halin2p3-00965620
halhal-00931476
10,halhal-00780833
halhal-00549940
halinria-00514441
halhal-01151533
20,halhal-00708674
30,halhal-02400730
halhal-02079225
40,halhal-01068827
halhal-01074519
hallirmm-02387999
halhal-00604338
50,halhal-01891375
halhal-00710069
nnt2021bord0246
nnt2013greni057
haltel-00541260
60,hallirmm-00806803
nnt2019bord0234
nnt2012grent049
nnt2020lysei131
nnt2012greni070
nnt2012greni097
nnt2015greai114
70,nnt2011grent037
nnt2022lysei029
nnt2019greai023
haltel-04524276
nnt2017greay088
80,nnt2015great120
nnt2015great119
nnt2018great050
nnt2014greni023
nnt2014greni043
nnt2016aixm4327
nnt2022gralt011
nnt2014pa112376
90,nnt2015enmp0015
nnt2015great021
nnt2011grent084
haltel-01255761
halhal-02545171
100,nnt2023bord0111
nnt2016great015
nnt2014greni004
nnt2018great096
nnt2010inpg0137
nnt2011grent125
nnt2013pa112132
nnt2012grent021
110,nnt2013tour4029
nnt2013aixm4373
120,130,140,150,160,170,halhal-02332427
halhal-04582810
halhal-00736000
halhal-00840093
n

In [36]:
import pickle

In [40]:
citations = {}
for e in oa_data:
    citations[e['doi'].replace('https://doi.org/', '')] = e['counts_by_year']

In [42]:
len(citations)

284

In [55]:
citations['10.1016/j.mee.2022.111875']

[{'year': 2024, 'cited_by_count': 1}]

In [45]:
import json

In [47]:
json.dump(citations, open('citations_3d_integration.json', 'w'), indent=2)

In [49]:
network = json.load(open('/Users/eric/Downloads/3d_network_authors.json', 'r'))

In [52]:
clusters = network['clusters']

In [62]:
for c in clusters:
    current_citations = {}
    for y in range(2012, 2025):
        current_citations[y] = 0
    for p in c['publications']:
        if p['id'].replace('doi', '') in citations:
            for cit in citations[p['id'].replace('doi', '')]:
                if cit['year'] in current_citations:
                    current_citations[cit['year']] += cit['cited_by_count']
    c['cited_by_count'] = current_citations

In [77]:
for c in clusters:
    print(c['label'])
    nb_publis = len(c['publications'])
    c['nb_publis'] = nb_publis
    print(f'{nb_publis} publications')
    nb_citations_2024_by_publi = c['cited_by_count'][2024]/nb_publis
    print(f'ratio {nb_citations_2024_by_publi}')
    c['ratio'] = nb_citations_2024_by_publi
    print(c['cited_by_count'])
    print('______')

3D Integration and Materials Engineering
26 publications
ratio 0.6923076923076923
{2012: 0, 2013: 1, 2014: 3, 2015: 17, 2016: 23, 2017: 13, 2018: 24, 2019: 39, 2020: 19, 2021: 38, 2022: 25, 2023: 24, 2024: 18}
______
Advanced 3D CMOS and Interconnects
41 publications
ratio 0.4878048780487805
{2012: 10, 2013: 25, 2014: 35, 2015: 52, 2016: 58, 2017: 48, 2018: 48, 2019: 54, 2020: 36, 2021: 57, 2022: 44, 2023: 59, 2024: 20}
______
3D Integration for Charged Particle Detection
18 publications
ratio 0.16666666666666666
{2012: 5, 2013: 3, 2014: 5, 2015: 10, 2016: 7, 2017: 7, 2018: 7, 2019: 15, 2020: 7, 2021: 8, 2022: 14, 2023: 8, 2024: 3}
______
Reliability and Hybrid Integration
23 publications
ratio 0.782608695652174
{2012: 1, 2013: 6, 2014: 8, 2015: 9, 2016: 15, 2017: 7, 2018: 5, 2019: 17, 2020: 14, 2021: 15, 2022: 23, 2023: 34, 2024: 18}
______
3D Integration and X-ray Characterization
15 publications
ratio 0.26666666666666666
{2012: 5, 2013: 3, 2014: 12, 2015: 5, 2016: 6, 2017: 4, 2018: 

In [78]:
sorted_clusters = sorted(clusters, key=lambda d: d['ratio'], reverse=True)

In [79]:
pd.DataFrame(sorted_clusters)

,cluster,label,publications,cited_by_count,ratio,nb_publis
0,11,Integrated Circuits and Photonic Applications,"[{'id': 'doi10.1109/ectc.2019.00249', 'title':...","{2012: 0, 2013: 0, 2014: 0, 2015: 0, 2016: 8, ...",0.800000,5
1,5,Direct Bonding and Integrated Circuit Security,"[{'id': 'doi10.4071/isom-2016-tp21', 'title': ...","{2012: 1, 2013: 4, 2014: 1, 2015: 3, 2016: 1, ...",0.785714,14
2,7,Reliability and Hybrid Integration,"[{'id': 'halhal-00931476', 'title': 'Investiga...","{2012: 1, 2013: 6, 2014: 8, 2015: 9, 2016: 15,...",0.782609,23
3,4,3D Integration and Materials Engineering,"[{'id': 'doi10.1016/j.mee.2022.111875', 'title...","{2012: 0, 2013: 1, 2014: 3, 2015: 17, 2016: 23...",0.692308,26
4,13,Nanoelectronics and Quantum Devices,"[{'id': 'halhal-01891375', 'title': 'Toward 3D...","{2012: 0, 2013: 0, 2014: 0, 2015: 0, 2016: 1, ...",0.571429,7
5,2,Advanced 3D CMOS and Interconnects,"[{'id': 'doi10.7873/date.2013.166', 'title': '...","{2012: 10, 2013: 25, 2014: 35, 2015: 52, 2016:...",0.487805,41
6,18,Non-volatile Memory and Integration,"[{'id': 'doi10.1364/oe.18.016162', 'title': '3...","{2012: 1, 2013: 0, 2014: 3, 2015: 6, 2016: 4, ...",0.400000,10
7,6,Low Temperature and High Density Integration,"[{'id': 'doi10.1109/vlsi-tsa.2019.8804665', 't...","{2012: 0, 2013: 0, 2014: 1, 2015: 1, 2016: 0, ...",0.400000,5
8,1,3D Integration and Power Supply,"[{'id': 'hallirmm-00806803', 'title': 'Resisti...","{2012: 0, 2013: 1, 2014: 5, 2015: 1, 2016: 6, ...",0.272727,11
9,8,3D Integration and X-ray Characterization,"[{'id': 'doi10.4071/2380-4505-2018.1.000718', ...","{2012: 5, 2013: 3, 2014: 12, 2015: 5, 2016: 6,...",0.266667,15


In [80]:
json.dump(sorted_clusters, open('clusters_3d', 'w'), indent=2)